# Assignment ONE
Scrape the website to load data into Dataframe per specifications

Load Libraries needed

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 16.7MB/s eta 0:00:01
Libraries imported.


In [102]:
from bs4 import BeautifulSoup as BS
import requests

# Initialize blank DataFrame to hold data

toronto_data=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood']) 

#Read the website
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website=requests.get(link).text

#Create BeautifulSoup object from the website html that holds the first table body
html=BS(website,'lxml')
html_file = html.tbody

# Iterate for each row in the table
for r,table_row in enumerate(html_file.find_all('tr')):
    if r > 0: # Ignor the first row as it has the column headings versus data
        for c, data in enumerate(table_row.find_all('td')): # Iterate across each row's columns to assign into dataframe
            if c == 0:
                toronto_data.loc[r,'PostalCode']=data.text
            elif c == 1:
                toronto_data.loc[r,'Borough']=data.text
            elif c == 2:
                toronto_data.loc[r,'Neighborhood']=data.text
toronto_data.replace(r'\n','',regex=True,inplace=True)   #### Delete the line return characters

#Drop the rows where Borough is Not assigned
indexBoroughs=toronto_data[toronto_data['Borough'] == 'Not assigned'].index
toronto_data.drop(indexBoroughs,axis=0,inplace=True)

#Change / in Neighborhoods to be a ,
toronto_data.replace(r'/',',',regex=True,inplace=True) 

# Reindex to set first row index = 1
toronto_data.reset_index(drop=True,inplace=True) 
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [103]:
toronto_data.shape

(103, 3)

# Assignment TWO
Postal Code Latitude and Longitude Corrdinates

Tried the loading via Geocoder but it would not return answers, so switched to the file and join approach

In [104]:
geo_data=pd.read_csv('https://cocl.us/Geospatial_data')
toronto_data=toronto_data.join(geo_data.set_index('Postal Code'), on='PostalCode')

In [105]:
print(toronto_data.shape)
print("Number of Unique Neighborhoods in toronto_data is:",toronto_data['Neighborhood'].nunique())
toronto_data

(103, 5)
Number of Unique Neighborhoods in toronto_data is: 98


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Assignment THREE
## Cluster Toronto

### Get the Foursquare Data for Toronto by Postal Code

In [146]:
{
    "tags": [
        "hide_input",
        "hide_output"
    ]
}
CLIENT_ID = 'LJ4UPISO5TQFFLIQI21AQDBB5YCE3Z2YCLHR3DPDCAAIWMLF' # your Foursquare ID
CLIENT_SECRET = 'VHB0HC0RZHP4A2MA2SRYXFRFNP1GP0ZYAIPMKN1HY1BY3PC0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [107]:
# Create the function to get the nearby venues for each Neighborhood with a limit of 500 venues per Neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 500
    venues_list=[]
    count=1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name," ",count)
        count=count+1
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    return(nearby_venues)

# Call the function using all of the Neighborhoods in Toronto
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods   1
Victoria Village   2
Regent Park , Harbourfront   3
Lawrence Manor , Lawrence Heights   4
Queen's Park , Ontario Provincial Government   5
Islington Avenue   6
Malvern , Rouge   7
Don Mills   8
Parkview Hill , Woodbine Gardens   9
Garden District, Ryerson   10
Glencairn   11
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale   12
Rouge Hill , Port Union , Highland Creek   13
Don Mills   14
Woodbine Heights   15
St. James Town   16
Humewood-Cedarvale   17
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood   18
Guildwood , Morningside , West Hill   19
The Beaches   20
Berczy Park   21
Caledonia-Fairbanks   22
Woburn   23
Leaside   24
Central Bay Street   25
Christie   26
Cedarbrae   27
Hillcrest Village   28
Bathurst Manor , Wilson Heights , Downsview North   29
Thorncliffe Park   30
Richmond , Adelaide , King   31
Dufferin , Dovercourt Village   32
Scarborough Village   33
Fairview , Henry Farm , Oriole   34
Northwood Park , York Unive

In [108]:
# Determine the number of venues and visualize the resulting dataframe
print(toronto_venues.shape)
print("The number of unique neighborhoods in toronto_venues is:",toronto_venues['Neighborhood'].nunique())
toronto_venues.head()

(2173, 7)
The number of unique neighborhoods in toronto_venues is: 95


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### From the above we can see the toronto_data had 98 unique neighborhoods, but toronto_venues only has 95 so 3 neighborhoods in toronto_data did not have any venue information returned.  Will need to delete those neighborhoods from toronto_data

In [109]:
# Find the Unique neighborhoods in tornonto_data that are not in tornonto_venues and drop them from toronto_data

## define a function to determine not in both lists and return as a list
def setdiff_sorted(array1,array2,assume_unique=False):
    ans = np.setdiff1d(array1,array2,assume_unique).tolist()
    if assume_unique:
        return sorted(ans)
    return ans


drop_neighs=setdiff_sorted(toronto_data['Neighborhood'],toronto_venues['Neighborhood'])

## Drop those neighborhoods from toronto_data
indexNeighs=[]
for neigh in drop_neighs:
    indexNeighs.append(toronto_data[toronto_data['Neighborhood'] == neigh].index)
indexNeighs=list(itertools.chain.from_iterable(indexNeighs))
toronto_data.drop(indexNeighs,axis=0,inplace=True)

In [110]:
# Verify the unique neighborhoods in both 

print("The number of unique neighborhoods in toronto_data is:",toronto_data['Neighborhood'].nunique())
print("The number of unique neighborhoods in toronto_venues is:",toronto_venues['Neighborhood'].nunique())
any_left=setdiff_sorted(toronto_data['Neighborhood'],toronto_venues['Neighborhood'])
print("The uniqu neighborhoods in toronto_data not in toronto_venues is: ",any_left)

The number of unique neighborhoods in toronto_data is: 95
The number of unique neighborhoods in toronto_venues is: 95
The uniqu neighborhoods in toronto_data not in toronto_venues is:  []


In [111]:
# See the number of venues per neighborhood
toronto_venues.groupby(by='Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood , Long Branch",11,11,11,11,11,11
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17


### Drop Neighborhoods with less than 25% of the mean number of venues, as these are not comparable neighborhoods

In [112]:
# Find the Neighborhoods where the Venue is Less than 25% of the Mean
## Group by neighborhood and count the number of venues
venues_grouped = toronto_venues.groupby(by='Neighborhood').count()
## Calculate half the mean as an interger
half_mean=.25*venues_grouped['Venue'].mean()
half_mean=int(round(half_mean,0))
half_mean
## Find the neighborhoods with less than half the mean number of venues
VenuesNeigh=venues_grouped[venues_grouped['Venue'] < half_mean]
VenuesNeigh.reset_index(inplace=True)
VenuesNeigh=VenuesNeigh['Neighborhood']
## Drop the rows in toronto_venues where the neighborhood is on of those where venues is less than half the mean
### Determine the indexs in toronto_venues where the condition is true so you can drop those rows
import itertools
indexVenues=[]
indexData=[]
for neigh in VenuesNeigh: 
    indexVenues.append(toronto_venues[toronto_venues['Neighborhood'] == neigh].index)
    indexData.append(toronto_data[toronto_data['Neighborhood'] == neigh].index)
indexVenues=list(itertools.chain.from_iterable(indexVenues))
indexData=list(itertools.chain.from_iterable(indexData))

### Drop rows with indexs in above list
toronto_venues.drop(indexVenues,axis=0,inplace=True)
toronto_venues.reset_index(drop=True,inplace=True)
toronto_data.drop(indexData,axis=0,inplace=True)
toronto_data.reset_index(drop=True,inplace=True)

print("Toronto_venues shape is:",toronto_venues.shape)

# Verify the unique neighborhoods in both are still the same number
print("The number of unique neighborhoods in toronto_data is:",toronto_data['Neighborhood'].nunique())
print("The number of unique neighborhoods in toronto_venues is:",toronto_venues['Neighborhood'].nunique())
any_left=setdiff_sorted(toronto_data['Neighborhood'],toronto_venues['Neighborhood'])
print("The uniqu neighborhoods in toronto_data not in toronto_venues is: ",any_left)

# Visualize toronto_venues
toronto_venues.head()

Toronto_venues shape is: (2064, 7)
The number of unique neighborhoods in toronto_data is: 60
The number of unique neighborhoods in toronto_venues is: 60
The uniqu neighborhoods in toronto_data not in toronto_venues is:  []


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [113]:
# Group by Neighborhood by One Hot Coding by Normalized Venue Category

# one hot encoding by Venue Category
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Group by Neighborhood with the mean of each Venue Category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Alderwood , Long Branch",0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000

In [114]:
toronto_grouped.head()
toronto_grouped.shape

(60, 253)

### Find the top 10 Venues for Each

In [115]:
# Define a function to sort and keep only the specified top number of venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood , Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Skating Rink,Coffee Shop
1,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Shopping Mall,Bridal Shop,Supermarket,Sushi Restaurant,Deli / Bodega,Middle Eastern Restaurant,Restaurant,Fried Chicken Joint
2,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Juice Bar,Pizza Place,Fast Food Restaurant,Indian Restaurant,Butcher,Pub
3,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Italian Restaurant,Farmers Market,Beer Bar,Seafood Restaurant
4,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
5,Business reply mail Processing CentrE,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
6,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden
7,Canada Post Gateway Processing Centre,Coffee Shop,Intersection,Hotel,Gym,American Restaurant,Mediterranean Restaurant,Sandwich Place,Burrito Place,Middle Eastern Restaurant,Fried Chicken Joint
8,Cedarbrae,Lounge,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Hakka Restaurant,Bakery,Doner Restaurant
9,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Ice Cream Shop,Thai Restaurant,Spa,Burger Joint,Bubble Tea Shop


### Perform K-Means Clustering on Toronto Neighborhoods

In [116]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 8, 1, 0, 1, 0, 0, 1, 2, 6, 6, 1, 1,
       1, 1, 5, 9, 1, 1, 0, 4, 1, 1, 1, 1, 6, 6, 1, 1, 1, 0, 1, 1, 1, 1,
       6, 1, 1, 6, 1, 1, 1, 0, 1, 6, 1, 1, 0, 7, 1, 3], dtype=int32)

### Add the Labels to the Neighborhood

In [117]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Theater,Café,Electronics Store,Distribution Center
1,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Accessories Store,Arts & Crafts Store,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
2,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Sandwich Place,Mexican Restaurant,Juice Bar,Hobby Shop,Fried Chicken Joint,Distribution Center,Creperie,College Auditorium
3,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Asian Restaurant,Dim Sum Restaurant,Supermarket,Café,Sporting Goods Shop
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,0,Pizza Place,Fast Food Restaurant,Pharmacy,Intersection,Bus Line,Athletics & Sports,Gastropub,Bank,Pet Store,Gym / Fitness Center


In [118]:
# Several Neighborhoods have NaN Cluster Labels Due to being outliers

toronto_merged.dropna(axis=0,inplace=True)

# Change Cluster Labels back to integer data type (cast as float b/c had NaN values)
toronto_merged.astype({'Cluster Labels':'int64'},inplace=True)

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Theater,Café,Electronics Store,Distribution Center
1,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Accessories Store,Arts & Crafts Store,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
2,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Sandwich Place,Mexican Restaurant,Juice Bar,Hobby Shop,Fried Chicken Joint,Distribution Center,Creperie,College Auditorium
3,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Asian Restaurant,Dim Sum Restaurant,Supermarket,Café,Sporting Goods Shop
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,0,Pizza Place,Fast Food Restaurant,Pharmacy,Intersection,Bus Line,Athletics & Sports,Gastropub,Bank,Pet Store,Gym / Fitness Center


### Map the Clusters with Lales

In [119]:
# Get the Longitude and Latitude for Toronto, CA

city = 'Toronto'
st= 'CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city,st)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of ',city,st,' are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of  Toronto CA  are 43.6534817, -79.3839347.


In [120]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [135]:
i=0
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 0 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East York,0,Pizza Place,Fast Food Restaurant,Pharmacy,Intersection,Bus Line,Athletics & Sports,Gastropub,Bank,Pet Store,Gym / Fitness Center
31,East Toronto,0,Pizza Place,Gym,Sandwich Place,Ice Cream Shop,Steakhouse,Fast Food Restaurant,Italian Restaurant,Light Rail Station,Liquor Store,Brewery
35,North York,0,Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Juice Bar,Pizza Place,Fast Food Restaurant,Indian Restaurant,Butcher,Pub
39,Central Toronto,0,Gym,Asian Restaurant,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Hotel,Park,Gastropub,Dumpling Restaurant
41,Etobicoke,0,Pizza Place,Coffee Shop,Chinese Restaurant,Intersection,Sandwich Place,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
45,Central Toronto,0,Sandwich Place,Café,Coffee Shop,BBQ Joint,Pub,Middle Eastern Restaurant,Donut Shop,Burger Joint,History Museum,Pharmacy
48,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Coffee Shop,Greek Restaurant,Seafood Restaurant
51,Scarborough,0,Pizza Place,Shopping Mall,Convenience Store,Intersection,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Bank,Chinese Restaurant
59,Etobicoke,0,Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool,Pub,Dance Studio,Sandwich Place,Skating Rink,Coffee Shop


In [136]:
i=1
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 1 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Theater,Café,Electronics Store,Distribution Center
1,North York,1,Furniture / Home Store,Clothing Store,Accessories Store,Arts & Crafts Store,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
2,Downtown Toronto,1,Coffee Shop,Diner,Sandwich Place,Mexican Restaurant,Juice Bar,Hobby Shop,Fried Chicken Joint,Distribution Center,Creperie,College Auditorium
3,North York,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Asian Restaurant,Dim Sum Restaurant,Supermarket,Café,Sporting Goods Shop
5,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Restaurant
6,North York,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Asian Restaurant,Dim Sum Restaurant,Supermarket,Café,Sporting Goods Shop
8,Downtown Toronto,1,Café,Coffee Shop,Cocktail Bar,Beer Bar,American Restaurant,Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park
9,Etobicoke,1,Park,Café,Coffee Shop,Beer Store,Pizza Place,Liquor Store,Cosmetics Shop,Yoga Studio,Distribution Center,Dim Sum Restaurant
11,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Italian Restaurant,Farmers Market,Beer Bar,Seafood Restaurant
12,East York,1,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Bank,Breakfast Spot,Supermarket,Fish & Chips Shop,Sports Bar,Liquor Store


In [137]:
i=2
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 2 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Scarborough,2,Indian Restaurant,Pet Store,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Yoga Studio,Diner,Deli / Bodega,Department Store,Dessert Shop


In [138]:
i=3
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 3 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East York,3,Skating Rink,Park,Cosmetics Shop,Beer Store,Dance Studio,Curling Ice,Pharmacy,Comfort Food Restaurant,Dessert Shop,Eastern European Restaurant


In [139]:
i=4
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 4 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Scarborough,4,Discount Store,Chinese Restaurant,Convenience Store,Department Store,Bus Station,Coffee Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


In [140]:
i=5
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 5 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Scarborough,5,Bus Line,Park,Ice Cream Shop,Bus Station,Metro Station,Intersection,Bakery,Soccer Field,Gas Station,Deli / Bodega


In [141]:
i=6
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 6 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,East York,6,Indian Restaurant,Yoga Studio,Supermarket,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station,Grocery Store
19,West Toronto,6,Pharmacy,Bakery,Park,Brewery,Bar,Supermarket,Bank,Café,Middle Eastern Restaurant,Music Venue
25,North York,6,Park,Hotel,Athletics & Sports,Liquor Store,Korean Restaurant,Discount Store,Food Truck,Baseball Field,Bank,Home Service
30,North York,6,Park,Hotel,Athletics & Sports,Liquor Store,Korean Restaurant,Discount Store,Food Truck,Baseball Field,Bank,Home Service
33,North York,6,Park,Hotel,Athletics & Sports,Liquor Store,Korean Restaurant,Discount Store,Food Truck,Baseball Field,Bank,Home Service
37,North York,6,Park,Hotel,Athletics & Sports,Liquor Store,Korean Restaurant,Discount Store,Food Truck,Baseball Field,Bank,Home Service
55,Etobicoke,6,Gym,Fast Food Restaurant,Café,Restaurant,Liquor Store,Fried Chicken Joint,Hobby Shop,American Restaurant,Bakery,Pizza Place
56,Etobicoke,6,Grocery Store,Liquor Store,Video Store,Beer Store,Pharmacy,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Sandwich Place,Japanese Restaurant
57,Scarborough,6,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Bank,Coffee Shop,Supermarket,Breakfast Spot,Pharmacy,Pizza Place
64,Etobicoke,6,Gym,Convenience Store,Bakery,Tanning Salon,Burger Joint,Discount Store,Sandwich Place,Supplement Shop,Social Club,Hardware Store


In [142]:
i=7
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 7 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Scarborough,7,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Bakery,Breakfast Spot,Sandwich Place,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner


In [143]:
i=8
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 8 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,8,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Nightclub,Baby Store,Coffee Shop,Restaurant,Diner


In [144]:
i=9
print("The Data for Cluster",i,"is:")
toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

The Data for Cluster 9 is:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,9,Rental Car Location,Mexican Restaurant,Breakfast Spot,Medical Center,Intersection,Bank,Electronics Store,Costume Shop,Coworking Space,Eastern European Restaurant
